# **Lesson01: Lexicon-based Methods**

## Download ChnSentiCorp dataset

In [9]:
import requests
import pandas as pd
from io import StringIO


request = requests.get(
    "https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv")
with open("ChnSentiCorp_htl_all.csv", "wb") as f:
  f.write(request.content)
    
df = pd.read_csv(StringIO(request.content.decode()), sep=",")
df.sample(10, random_state=0)

,label,review
1726,1,"房间还算干净整洁,服务也可以,以这个价格来说,不错了,建议大家要定有窗户的房间,但是餐厅不够..."
1085,1,"这里的早餐是我看到的最差的一个,基本上没什么吃的,就看到服务员在不听的加白粥,下次在来我是不..."
2404,1,房间设施还是很先进的，自动化程度相当高。但是我最喜欢的还是演艺馆的演出。竟然看了6场。水平真...
2368,1,住的的是两房一厅，卧室偏小，到卫生间要路过客厅和门厅，不太方便。服务尚可，早餐以欧式为主。入...
6337,0,"175元入住标房B。大厅和电梯里有异味。房间里桌子也没擦干净,洗浴间装潢等设备太旧。早饭很差..."
6175,0,我全说客观的，大家自己参考。1、酒店电话不能打外线，携程为什么不说明？2、拖鞋不是一次性的3...
3357,1,因为有了去年的经验，去之前我和酒店联系了一下，得知相比豪华商务间还是标准间相对安静，所以在携...
3443,1,酒店在东营当地是最高档的，服务人员训练有素，服务意识很好。酒店很新，房间也很干净，零点厅的饭...
509,1,酒店房间还算可以，对得起五星级的称号。早餐品种也算是丰富。不足之处酒店近着大马路，楼层太低晚...
2735,1,交通便利，酒店位于商业中心。前台服务好，房间整洁干净。下次还会入住。


### Check the data distribution

In [10]:
tmp = df.label.value_counts()
tmp.index = ['Positive', 'Negative']
tmp

Positive    5322
Negative    2444
Name: label, dtype: int64

### Generate Balance Data

In [11]:
import pandas as pd


seed = 42


def get_balance_corpus(corpus_size, corpus_pos, corpus_neg, seed=42):
  sample_size = corpus_size // 2
  pos_sample = corpus_pos.sample(
    sample_size, random_state=seed,
    replace=corpus_pos.shape[0]<sample_size)
  neg_sample = corpus_neg.sample(
    sample_size, random_state=seed,
    replace=corpus_neg.shape[0]<sample_size)
  pd_corpus_balance = pd.concat([pos_sample, neg_sample])
  return pd_corpus_balance

df = pd.read_csv('ChnSentiCorp_htl_all.csv')

pd_positive = df[df.label==1]
pd_negative = df[df.label==0]

ChnSentiCorp_htl_ba_6000 = get_balance_corpus(
  6000, pd_positive, pd_negative, seed=seed)

tmp = ChnSentiCorp_htl_ba_6000.label.value_counts()
tmp.index = ['Positive', 'Negative']
tmp

Positive    3000
Negative    3000
Name: label, dtype: int64

### Clean the dataset
remove newline or EOL signal

In [12]:
ChnSentiCorp_htl_ba_6000.review = ChnSentiCorp_htl_ba_6000.review.str.replace('\n', ' ')
ChnSentiCorp_htl_ba_6000.review = ChnSentiCorp_htl_ba_6000.review.str.replace('\r', ' ')
ChnSentiCorp_htl_ba_6000.review = ChnSentiCorp_htl_ba_6000.review.str.replace('\t', ' ')
ChnSentiCorp_htl_ba_6000.review = ChnSentiCorp_htl_ba_6000.review.str.replace('[^\w\s]', ' ')
ChnSentiCorp_htl_ba_6000.sample(10, random_state=seed)

,label,review
818,1,没有想象的离西湖近 年轻人走走也得10 15分钟 因为窗不是面对西湖的 所以景基本看不到西湖...
7384,0,住的标准单人间实在是小 房间里一张大床已经塞得满满的了 设施极其简陋除了电视机和烧水壶其他什...
2715,1,初到当地 在携程上订了一晚这家酒店 印象中原来认为新疆很偏僻 没想到酒店的环境及服务与内地的...
2984,1,酒店的床比一般的香港酒店大 双人房的床都很大 枕头超多超舒服 酒店正好在金钟的正上方 逛街很...
5770,0,在网上综合考虑后 并打电话给汉庭 汉庭说靠街边的窗户都换成双层窗户 隔音很好 也有停车场 汉...
3950,1,总体感觉很不错 酒店硬件和软件都还满意 前台服务生服务热情到位 房间安静 不嘈杂 总体房间还...
926,1,感觉还算可以的 不过没有香格里拉那么好 价格和协议的也没什么区别 所以还是去香格里拉较划算点
696,1,各方面都一般 只要期望值不太高就还可以
2943,1,这次来的比较晚 出去吃饭不太方便 就在酒店的餐厅吃的海鲜自助 还可以 不过感觉价格稍微偏高了...
422,1,我订的是行政房 但酒店给我免费升级到公寓房 是他们新的房子 房间设施 装修都非常不错 而且可...


## **Transform from Simplified Chinese to Tranditional Chinese (Optional)**


### install OpenCC

In [5]:
# !pip install OpenCC

### Transfer Chinese between zh-CN and zh-TW


In [6]:
# from opencc import OpenCC


# cc = OpenCC('s2twp')  # s2twp, tw2sp

# split_sign = '-split-'
# ChnSentiCorp_htl_ba_6000.review = ChnSentiCorp_htl_ba_6000.review.astype(str)
# converted = cc.convert(split_sign.join(ChnSentiCorp_htl_ba_6000['review'].values))

# ChnSentiCorp_htl_ba_6000['review_converted'] = converted.split(split_sign)
# ChnSentiCorp_htl_ba_6000.sample(10, random_state=seed)

## **Read the sentiment lexicon**
Using NRC sentiment dictionary from Canada

### Download NRC lexicon

In [13]:
import zipfile
from io import BytesIO


request = requests.get("http://sentiment.nrc.ca/lexicons-for-research/NRC-Emotion-Lexicon.zip")

z = zipfile.ZipFile(BytesIO(request.content))
z.extractall("NRC-Emotion-Lexicon")

### Read lexicon and do some clean process

In [14]:
sentiment_list = ['Negative', 'Positive']
emotion_list = ['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust']

nrc = pd.read_excel('NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx')

nrc = nrc[['Chinese (Simplified) (zh-CN)'] + sentiment_list + emotion_list]  # Chinese (Traditional) (zh-TW)
nrc.rename(columns={'Chinese (Simplified) (zh-CN)': 'word'}, inplace=True)

nrc = nrc.groupby('word').sum().reset_index()
nrc.sample(15, random_state=0)

,word,Negative,Positive,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
4676,改革者,0,1,0,0,0,0,0,0,0,0
2513,图书管理员,0,0,0,0,0,0,0,0,0,0
7635,结语,0,0,0,0,0,0,0,0,0,0
8781,豹,0,0,0,0,0,0,0,0,0,0
4741,教学,0,0,0,0,0,0,0,0,0,0
4842,新郎,0,1,0,1,0,0,1,0,0,1
6712,男子气,0,1,0,0,0,0,0,0,0,0
3776,心理学家,0,0,0,0,0,0,0,0,0,0
3194,寒意,0,0,0,0,0,0,0,0,0,0
1308,凉意,0,1,0,0,0,0,0,0,0,0


### Seperate sentiment and emotion dictionary

In [15]:
import numpy as np


nrc_sentiment = nrc[sentiment_list].copy().values
with np.errstate(divide='ignore', invalid='ignore'):
  nrc_sentiment = np.true_divide(nrc_sentiment, nrc_sentiment.sum(axis=1)[:, np.newaxis])
  nrc_sentiment[~np.isfinite(nrc_sentiment)] = 0

nrc_sentiment = dict(zip(nrc['word'], nrc_sentiment))

nrc_emotion = nrc[emotion_list].copy().values
with np.errstate(divide='ignore', invalid='ignore'):
  nrc_emotion = np.true_divide(nrc_emotion, nrc_emotion.sum(axis=1)[:, np.newaxis])
  nrc_emotion[~np.isfinite(nrc_emotion)] = 0

nrc_emotion = dict(zip(nrc['word'], nrc_emotion))

### Test the sentiment and emotion result of word

In [16]:
test_word = '一致'

try:
  nrc_sentiment[test_word]
  print('test word: ', test_word)
  print('sentiment [{}]: {}'.format('/'.join(sentiment_list), nrc_sentiment[test_word]))
  print('emotion [{}]: {}'.format('/'.join(emotion_list), nrc_emotion[test_word]))
except KeyError:
  print('test word {} not in the lexicon.'.format(test_word))

test word:  一致
sentiment [Negative/Positive]: [0. 1.]
emotion [Anger/Anticipation/Disgust/Fear/Joy/Sadness/Surprise/Trust]: [0. 0. 0. 0. 0. 0. 0. 1.]


## **Tokenize**

### Get vocabulary for jieba tokenizer

In [17]:
vocab = nrc['word'].values

### Initialize jieba for chinese tokenize also add vocabulary to help the tokenize result better

In [18]:
import jieba


for word in vocab:
  jieba.add_word(word)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.900 seconds.
Prefix dict has been built successfully.


### download stopword list to avoid useless word impact the result.

In [19]:
stopwords = []

request = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/hit_stopwords.txt")
stopwords += request.content.decode().split('\n')

request = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/baidu_stopwords.txt")
stopwords += request.content.decode().split('\n')

request = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
stopwords += request.content.decode().split('\n')

request = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/scu_stopwords.txt")
stopwords += request.content.decode().split('\n')

stopwords = list(set(stopwords) - set(vocab))

### Let tokenzie the data!

In [20]:
from tqdm.auto import tqdm


tqdm.pandas()

ChnSentiCorp_htl_ba_6000['cut'] = ChnSentiCorp_htl_ba_6000.review.progress_apply(
  lambda x: [w for w in jieba.lcut(x) if w not in stopwords])

ChnSentiCorp_htl_ba_6000.sample(10, random_state=seed)

,label,review,cut
818,1,没有想象的离西湖近 年轻人走走也得10 15分钟 因为窗不是面对西湖的 所以景基本看不到西湖...,"[没有, 想象, 西湖, 近, 年轻人, 走走, 10, 15, 分钟, 窗, 面对, 西湖..."
7384,0,住的标准单人间实在是小 房间里一张大床已经塞得满满的了 设施极其简陋除了电视机和烧水壶其他什...,"[住, 标准, 单人间, 实在, 小, 房间, 里, 一张, 大床, 塞得, 满满的, 设施..."
2715,1,初到当地 在携程上订了一晚这家酒店 印象中原来认为新疆很偏僻 没想到酒店的环境及服务与内地的...,"[初到, 携程, 上订, 一晚, 这家, 酒店, 印象, 中, 新疆, 偏僻, 没想到, 酒..."
2984,1,酒店的床比一般的香港酒店大 双人房的床都很大 枕头超多超舒服 酒店正好在金钟的正上方 逛街很...,"[酒店, 床, 比, 一般, 香港, 酒店, 大, 双人房, 床, 很大, 枕头, 超多超,..."
5770,0,在网上综合考虑后 并打电话给汉庭 汉庭说靠街边的窗户都换成双层窗户 隔音很好 也有停车场 汉...,"[网上, 综合, 后, 打电话, 给汉庭, 汉庭, 说, 靠, 街边, 窗户, 换成, 双层..."
3950,1,总体感觉很不错 酒店硬件和软件都还满意 前台服务生服务热情到位 房间安静 不嘈杂 总体房间还...,"[总体, 感觉, 不错, 酒店, 硬件, 和, 软件, 满意, 前台, 服务生, 服务, 热..."
926,1,感觉还算可以的 不过没有香格里拉那么好 价格和协议的也没什么区别 所以还是去香格里拉较划算点,"[感觉, 算, 没有, 香格里拉, 好, 价格, 和, 协议, 没什么, 区别, 香格里拉,..."
696,1,各方面都一般 只要期望值不太高就还可以,"[方面, 一般, 期望值, 不太高]"
2943,1,这次来的比较晚 出去吃饭不太方便 就在酒店的餐厅吃的海鲜自助 还可以 不过感觉价格稍微偏高了...,"[来, 比较, 晚, 吃饭, 不太, 方便, 酒店, 餐厅, 吃, 海鲜, 自助, 感觉, ..."
422,1,我订的是行政房 但酒店给我免费升级到公寓房 是他们新的房子 房间设施 装修都非常不错 而且可...,"[我订, 行政, 房, 酒店, 给, 免费, 升级, 公寓, 房, 新, 房子, 房间, 设..."


save dataset for the next session

In [21]:
ChnSentiCorp_htl_ba_6000.to_csv('ChnSentiCorp_htl_ba_6000_cutted.csv', index=False, compression='gzip')

## **Using Sklearn CountVectorizer to get the multi-hot encoder**

### Initialize a count vector object and feed the vocab as vocabulary

In [22]:
from sklearn.feature_extraction.text import CountVectorizer


cnt_vec = CountVectorizer(vocabulary=vocab)

### Transform the word list to multi-hot label vector and turn into original word list

In [23]:
if isinstance(ChnSentiCorp_htl_ba_6000.cut[0], list):
  ChnSentiCorp_htl_ba_6000.cut = ChnSentiCorp_htl_ba_6000.cut.apply(lambda x: ' '.join(x))

multi_hot = cnt_vec.transform(ChnSentiCorp_htl_ba_6000.cut)
ChnSentiCorp_htl_ba_6000['word'] = [vocab[np.argwhere(row > 0)[:, 1]] for row in multi_hot]

ChnSentiCorp_htl_ba_6000.sample(10, random_state=seed)

,label,review,cut,word
818,1,没有想象的离西湖近 年轻人走走也得10 15分钟 因为窗不是面对西湖的 所以景基本看不到西湖...,没有 想象 西湖 近 年轻人 走走 10 15 分钟 窗 面对 西湖 景 看不到 西湖 把头...,"[分钟, 勉强, 想象, 没有, 远处, 面对]"
7384,0,住的标准单人间实在是小 房间里一张大床已经塞得满满的了 设施极其简陋除了电视机和烧水壶其他什...,住 标准 单人间 实在 小 房间 里 一张 大床 塞得 满满的 设施 简陋 电视机 和 烧 ...,"[中等, 临街, 安全, 实际, 房间, 标准, 水壶, 水平, 没有, 电梯, 设施]"
2715,1,初到当地 在携程上订了一晚这家酒店 印象中原来认为新疆很偏僻 没想到酒店的环境及服务与内地的...,初到 携程 上订 一晚 这家 酒店 印象 中 新疆 偏僻 没想到 酒店 环境 服务 与 内地...,"[印象, 服务, 环境]"
2984,1,酒店的床比一般的香港酒店大 双人房的床都很大 枕头超多超舒服 酒店正好在金钟的正上方 逛街很...,酒店 床 比 一般 香港 酒店 大 双人房 床 很大 枕头 超多超 舒服 酒店 正好 金钟 ...,"[一般, 国内, 房间, 方便, 服务, 枕头, 特别, 直接, 细节, 缺乏, 金钟, 非常]"
5770,0,在网上综合考虑后 并打电话给汉庭 汉庭说靠街边的窗户都换成双层窗户 隔音很好 也有停车场 汉...,网上 综合 后 打电话 给汉庭 汉庭 说 靠 街边 窗户 换成 双层 窗户 隔音 好 停车场...,"[一般, 人员, 双层, 发票, 只是, 噪声, 对面, 广告, 开始, 感觉, 愤怒, 房..."
3950,1,总体感觉很不错 酒店硬件和软件都还满意 前台服务生服务热情到位 房间安静 不嘈杂 总体房间还...,总体 感觉 不错 酒店 硬件 和 软件 满意 前台 服务生 服务 热情 到位 房间 安静 嘈...,"[不同, 丰富, 入住, 可能, 同样, 品种, 嘈杂, 学院, 安静, 希望, 带来, 广..."
926,1,感觉还算可以的 不过没有香格里拉那么好 价格和协议的也没什么区别 所以还是去香格里拉较划算点,感觉 算 没有 香格里拉 好 价格 和 协议 没什么 区别 香格里拉 划算 点,"[区别, 协议, 感觉, 没有]"
696,1,各方面都一般 只要期望值不太高就还可以,方面 一般 期望值 不太高,"[一般, 方面]"
2943,1,这次来的比较晚 出去吃饭不太方便 就在酒店的餐厅吃的海鲜自助 还可以 不过感觉价格稍微偏高了...,来 比较 晚 吃饭 不太 方便 酒店 餐厅 吃 海鲜 自助 感觉 价格 稍微 偏高 一点 住...,"[感觉, 房间, 方便, 比较, 餐厅]"
422,1,我订的是行政房 但酒店给我免费升级到公寓房 是他们新的房子 房间设施 装修都非常不错 而且可...,我订 行政 房 酒店 给 免费 升级 公寓 房 新 房子 房间 设施 装修 非常 不错 延迟...,"[交通, 公寓, 升级, 喜欢, 延迟, 房间, 方便, 行政, 装修, 设施, 附近, 非..."


## Initialize "get result" function and transform the word to result value

### Define an object getLexicon to handle lexicon check

In [24]:
class getLexicon(object):
    
  def __init__(self, map_dict, map_list):
    self.map_dict = map_dict
    self.map_list = map_list
    self.fillna = np.zeros_like(list(self.map_dict.values())[0])

  def get_word_in_lexicon(self, x):
    if not isinstance(x, (list, np.ndarray)):
      x = eval(x)
    r = [self.map_dict[i][np.newaxis, :] 
        for i in x if i in self.map_dict and len(self.map_dict[i]) > 0]
    try:
      r = np.concatenate(r, axis=0).sum(0)
    except:
      r = self.fillna
    return r

  def get_result(self,):
    raise NotImplementedError("get result is not implemented!")

### Define two object to get lexicon result

In [25]:
from scipy.special import softmax


class getSentiment(getLexicon):
  
  def __init__(self, **kwargs):
    super().__init__(**kwargs)  # init getLexicon

  def get_result(self, x):
    r = self.get_word_in_lexicon(x)  # get word vector of a sentence

    if sum(r) == 0:
      return 'Neutral'
    elif r[0] == r[1]:
      return 'Neutral'
    elif r.argmax() == 0:
      return self.map_list[0]
    else:
      return self.map_list[1]


class getEmotion(getLexicon):
    
  def __init__(self, topk=3, **kwargs):
    super().__init__(**kwargs)  # init getLexicon
    self.topk = topk

  def get_result(self, x):
    r = self.get_word_in_lexicon(x)  # get word vector of a sentence
    r = softmax(r)
    r = [self.map_list[i]
        for i in r.argsort()[::-1][:(r > 1 / self.fillna.shape[0]).sum()]]
    return ', '.join(r[:self.topk])
  

In [26]:
get_sen_fn = getSentiment(map_dict=nrc_sentiment, map_list=sentiment_list)
get_emo_fn = getEmotion(map_dict=nrc_emotion, map_list=emotion_list)
ChnSentiCorp_htl_ba_6000['sentiment_predict'] = ChnSentiCorp_htl_ba_6000.word.apply(get_sen_fn.get_result)
ChnSentiCorp_htl_ba_6000['emotion_predict'] = ChnSentiCorp_htl_ba_6000.word.apply(get_emo_fn.get_result)

ChnSentiCorp_htl_ba_6000.label = ChnSentiCorp_htl_ba_6000.label.map({0: 'Negative', 1: 'Positive'})

result = ChnSentiCorp_htl_ba_6000[['review', 'label', 'sentiment_predict', 'emotion_predict']].copy()

### **Check result**

In [27]:
result.sample(15, random_state=seed)

,review,label,sentiment_predict,emotion_predict
818,没有想象的离西湖近 年轻人走走也得10 15分钟 因为窗不是面对西湖的 所以景基本看不到西湖...,Positive,Negative,Anger
7384,住的标准单人间实在是小 房间里一张大床已经塞得满满的了 设施极其简陋除了电视机和烧水壶其他什...,Negative,Positive,"Trust, Joy"
2715,初到当地 在携程上订了一晚这家酒店 印象中原来认为新疆很偏僻 没想到酒店的环境及服务与内地的...,Positive,Neutral,Trust
2984,酒店的床比一般的香港酒店大 双人房的床都很大 枕头超多超舒服 酒店正好在金钟的正上方 逛街很...,Positive,Positive,"Trust, Sadness, Surprise"
5770,在网上综合考虑后 并打电话给汉庭 汉庭说靠街边的窗户都换成双层窗户 隔音很好 也有停车场 汉...,Negative,Negative,"Trust, Anticipation"
3950,总体感觉很不错 酒店硬件和软件都还满意 前台服务生服务热情到位 房间安静 不嘈杂 总体房间还...,Positive,Positive,Trust
926,感觉还算可以的 不过没有香格里拉那么好 价格和协议的也没什么区别 所以还是去香格里拉较划算点,Positive,Neutral,Trust
696,各方面都一般 只要期望值不太高就还可以,Positive,Positive,Trust
2943,这次来的比较晚 出去吃饭不太方便 就在酒店的餐厅吃的海鲜自助 还可以 不过感觉价格稍微偏高了...,Positive,Positive,"Trust, Surprise, Sadness"
422,我订的是行政房 但酒店给我免费升级到公寓房 是他们新的房子 房间设施 装修都非常不错 而且可...,Positive,Positive,"Joy, Sadness, Anger"


### check correct answer result

In [28]:
result[(result.label == result.sentiment_predict) & (result.emotion_predict.str.len() > 0)].sample(15, random_state=seed)

,review,label,sentiment_predict,emotion_predict
4578,还不错 经济实惠 条件也不错 反正我习惯这补充点评2008年4月10日 补充下 靠街的房间有...,Positive,Positive,"Trust, Anticipation"
5377,太差 脏 差 我住过的最不好的酒店 以后决不住了,Negative,Negative,"Trust, Surprise, Sadness"
1207,朋友推荐的宾馆 位置方便 价格合理 高楼的房间面积很宽敞 前台登记和退房时间很快 服务感觉不...,Positive,Positive,Trust
5331,房间的环境非常差 而且房间还不隔音 住的不舒服,Negative,Negative,Sadness
3858,全家去千岛湖旅行 住了3家洒店 千岛湖 杭州 湖州各住一晚 同是三星 假日的硬件设施最逊 我...,Positive,Positive,"Joy, Anticipation, Trust"
3765,不是特别满意位置不太好但价格比较合理去了才知道 他是3星级的 外表有点旧旧的,Positive,Positive,Joy
1378,性价比非常好的酒店 房间设施超好 卫生间分了3个部分 淋浴房 卫生间 化妆间 非常干净 装修...,Positive,Positive,"Sadness, Joy, Anticipation"
6019,服务非常差 环境不好 建议别去 提前一个星期就定好了 并且定了加床 到了前台告诉我们能加闯的...,Negative,Negative,"Trust, Sadness"
383,房间干净整齐 服务员态度很好 不过餐厅一般 给我的早餐券上的套餐与餐厅的套餐不符,Positive,Positive,Trust
3383,酒店的地理位置优越 适合办公人事入住 服务也比较到位 下次会继续考虑,Positive,Positive,"Trust, Anger"


### check wrong answer result

In [ ]:
result[(result.label != result.sentiment_predict) & (result.emotion_predict.str.len() > 0)].sample(15, random_state=seed)

## **Define Metric Object**

In [29]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support


class Metric(object):

  def __init__(self, y_true):
    self.y_true = y_true
      
  def get_metric(self, y_pred, y_true=None):
    if y_true is None:
      y_true = self.y_true
    
    loss = np.nan
    if y_pred.ndim > 1:
      loss = log_loss(y_true=y_true, y_pred=y_pred)
      y_pred = y_pred.argmax(axis=1)
    
    acuuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    precision, recall, f_score, _ = precision_recall_fscore_support(
      y_true=y_true, y_pred=y_pred, average='macro', zero_division='warn')
    
    print('loss:', loss)
    print('acuuracy:', acuuracy)
    print('precision:', precision)
    print('recall:', recall)
    print('f_score:', f_score)


sentiment_result_map = {'Positive': 1, 'Negative': 0}
metric_fn = Metric(y_true=ChnSentiCorp_htl_ba_6000.label.map(sentiment_result_map))

### Transfer Neutral as opposite result and trans

In [30]:
pred = result[['label', 'sentiment_predict']].copy()

neutral_index = (pred.sentiment_predict == 'Neutral')

opposite_transform = lambda x: list(set(['Negative', 'Positive']) - set([x]))[0]
pred.loc[neutral_index, 'sentiment_predict'] = pred.loc[neutral_index, 'label'].apply(opposite_transform)

pred = pred.sentiment_predict.map(sentiment_result_map)

### Get metric result

In [31]:
metric_fn.get_metric(pred)

loss: nan
acuuracy: 0.4706666666666667
precision: 0.46035500292832365
recall: 0.4706666666666667
f_score: 0.4338529550700999
